In [1]:
import time
import logging
from dotenv import load_dotenv
from llama_parse import LlamaParse
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings, VectorStoreIndex
from llama_index.core.response.pprint_utils import pprint_response
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.indices.postprocessor import SimilarityPostprocessor
from llama_index.llms.openai import OpenAI
from llama_index.llms.groq import Groq
from llama_index.vector_stores.pinecone import PineconeVectorStore
from pinecone.grpc import PineconeGRPC
from pinecone import ServerlessSpec
import google.generativeai as genai
import os

d:\vasu_Installed_applications\anaconda3-2024.10-1\envs\rag_pinecone_mongo\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
class APIKeysManager:
    """Manages API keys for the application."""

    def __init__(self):
        load_dotenv()
        self.keys = {
            "OPENAI_API_KEY": os.getenv("OPENAI_API_KEY"),
            "PINECONE_API_KEY": os.getenv("PINECONE_API_KEY"),
            "LLAMAPARSER_API_KEY": os.getenv("LLAMAPARSER_API_KEY"),
            "GROQ_API_KEY": os.getenv("GROQ_API_KEY"),
            "GEMINI_API_KEY": os.getenv("GEMINI_API_KEY"),
        }
        self.validate_keys()

    def validate_keys(self):
        missing_keys = [key for key, value in self.keys.items() if not value]
        if missing_keys:
            raise ValueError(f"Missing API keys: {', '.join(missing_keys)}")

    def get_key(self, key_name):
        return self.keys.get(key_name)



In [3]:
class PineconeManager:
    """Manages Pinecone interactions."""

    def __init__(self, api_key, index_name, dimension=1536, metric="cosine", region="us-east-1"):
        self.api_key = api_key
        self.index_name = index_name
        self.dimension = dimension
        self.metric = metric
        self.region = region
        self.client = PineconeGRPC(api_key=api_key)
        self.index = self._initialize_index()

    def _initialize_index(self):
        print("Initializing Pinecone index...")
        if self.index_name not in self.client.list_indexes().names():
            self.client.create_index(
                name=self.index_name,
                dimension=self.dimension,
                metric=self.metric,
                spec=ServerlessSpec(cloud="aws", region=self.region),
            )
            while not self.client.describe_index(self.index_name).status["ready"]:
                time.sleep(1)
        return self.client.Index(self.index_name)

    def get_vector_store(self):
        return PineconeVectorStore(pinecone_index=self.index)

In [4]:
class QueryEngineManager:
    """Manages query engine setup and operations."""

    def __init__(self, pinecone_manager, embedding_model):
        self.vector_store = pinecone_manager.get_vector_store()
        self.vector_index = VectorStoreIndex.from_vector_store(vector_store=self.vector_store)
        self.retriever = VectorIndexRetriever(index=self.vector_index, similarity_top_k=7)
        self.postprocessor = SimilarityPostprocessor(similarity_cutoff=0.3)
        self.query_engine = RetrieverQueryEngine(
            retriever=self.retriever, node_postprocessors=[self.postprocessor]
        )

    def query(self, query):
        print("Running query on the query engine...")
        return self.query_engine.query(query)

In [5]:
class DocumentManager:
    """Manages document loading, splitting, and embedding."""

    def __init__(self, llama_parser_api_key, embedding_model):
        self.loader = LlamaParse(api_key=llama_parser_api_key, result_type="markdown", verbose=True)
        self.splitter = SentenceSplitter(include_metadata=True)
        self.embedding_model = embedding_model

    def process_documents(self, input_files):
        print("Loading and processing documents...")
        documents = self.loader.load_data(file_path=input_files)
        nodes = self.splitter.get_nodes_from_documents(documents)
        for node in nodes:
            if node.text:
                node.embedding = self.embedding_model.get_text_embedding(node.get_content(metadata_mode="all"))
        return nodes

In [7]:
class QueryRewriter:
    """Handles query rewriting based on history."""

    def __init__(self, openai_api_key):
        self.llm = OpenAI(model="gpt-3.5-turbo", api_key=openai_api_key)

    def rewrite_query(self, query, history=""):
        query_prompt = f"""
            ### Instruction: Rewrite the query

            The original query is as follows: {query}
            We have provided an existing history: {history}
            
            Rewrite the query into a clear and specific form suitable for a vector database search.
            - Preserve the original meaning.
            - Use history only if necessary.
        """
        rewritten_query = self.llm.complete(query_prompt)
        return rewritten_query.text


In [18]:
# Simle RAG when we have single vector store

class RAGApplication:
    """Main application class to manage the entire workflow."""

    def __init__(self):
        print("Initializing API keyss")
        self.api_keys = APIKeysManager()
        print("API Keys verified")
        self.embedding_model = OpenAIEmbedding(model="text-embedding-3-small")
        print("Embedding model initialized")
        Settings.embed_model = self.embedding_model
        print("Embedding model set")
        self.pinecone_manager = PineconeManager(
            api_key=self.api_keys.get_key("PINECONE_API_KEY"),
            index_name="capston-llama-parser"
        )
        print("Pinecone manager initialized")
        self.document_manager = DocumentManager(
            llama_parser_api_key=self.api_keys.get_key("LLAMAPARSER_API_KEY"),
            embedding_model=self.embedding_model,
        )
        print("Document manager initialized")
        self.query_engine_manager = QueryEngineManager(
            pinecone_manager=self.pinecone_manager, embedding_model=self.embedding_model
        )
        print("Query engine manager initialized")
        self.query_rewriter = QueryRewriter(openai_api_key=self.api_keys.get_key("OPENAI_API_KEY"))
        print("Query rewriter initialized")

    def run_query(self, query, history=""):
        relavent_documents = []
        rewritten_query = self.query_rewriter.rewrite_query(query, history)
        response = self.query_engine_manager.query(rewritten_query)
        pprint_response(response, show_source=True)
        for node in response.source_nodes:
            relavent_documents.append(node.text)
        return relavent_documents, rewritten_query

    def upload_documents(self, input_files):
        print("Uploading documents...")
        nodes = self.document_manager.process_documents(input_files)
        vector_store = self.pinecone_manager.get_vector_store()
        vector_store.add(nodes=nodes)
        print("Documents uploaded successfully.")


In [19]:
class MultiIndexManager:
    """Manages multiple Pinecone indices for different contexts."""

    def __init__(self, api_key, indices_info):
        self.api_key = api_key
        self.indices = {}
        self.client = PineconeGRPC(api_key=api_key)

        for index_name, config in indices_info.items():
            if index_name not in self.client.list_indexes().names():
                self.client.create_index(
                    name=index_name,
                    dimension=config['dimension'],
                    metric=config['metric'],
                    spec=ServerlessSpec(cloud="aws", region=config['region']),
                )
            self.indices[index_name] = self.client.Index(index_name)

    def get_vector_store(self, index_name):
        return PineconeVectorStore(pinecone_index=self.indices[index_name])

In [88]:
class ContextAugmentedQueryEngine:
    """Manages query processing across multiple indices."""

    def __init__(self, multi_index_manager, embedding_model):
        self.multi_index_manager = multi_index_manager
        self.embedding_model = embedding_model

    def query_index(self, index_name, query, similarity_cutoff=0.3):
        vector_store = self.multi_index_manager.get_vector_store(index_name)
        vector_index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
        retriever = VectorIndexRetriever(index=vector_index, similarity_top_k=7)
        postprocessor = SimilarityPostprocessor(similarity_cutoff=similarity_cutoff)
        query_engine = RetrieverQueryEngine(
            retriever=retriever, node_postprocessors=[postprocessor]
        )
        return query_engine.query(query)

    def query_all_indices(self, query, similarity_cutoff=0.3):
        results = {} 
        for index_name in self.multi_index_manager.indices.keys():
            if index_name.split("-")[0] in query.lower():
                print(f"Querying index: {index_name}")
                results[index_name] = self.query_index(index_name, query, similarity_cutoff)
        return results

In [110]:
class RAGApplicationWithAgents(RAGApplication):
    """Extended RAG application with multiple indices and OpenAI function-calling agents."""

    def __init__(self):
        super().__init__()

        indices_info = {
            "uber-llama-parser": {"dimension": 1536, "metric": "cosine", "region": "us-east-1"},
            "infosys-llama-parser": {"dimension": 1536, "metric": "cosine", "region": "us-east-1"},
            "alteryx-llama-parser": {"dimension": 1536, "metric": "cosine", "region": "us-east-1"},
        }

        self.multi_index_manager = MultiIndexManager(
            api_key=self.api_keys.get_key("PINECONE_API_KEY"), indices_info=indices_info
        )
        self.context_query_engine = ContextAugmentedQueryEngine(
            multi_index_manager=self.multi_index_manager, embedding_model=self.embedding_model
        )

    def run_context_aware_query(self, query):
        rewritten_query = self.query_rewriter.rewrite_query(query)
        print("Rewritten_query", rewritten_query)
        results = self.context_query_engine.query_all_indices(rewritten_query)
        # pprint_response(results, show_source=True)
        aggregated_results = {}
        for index, response in results.items():
            documents = [node.text for node in response.source_nodes]
            metadata = [node.metadata for node in response.source_nodes]    
            aggregated_results[index] = {
                "documents": documents,
                "response": response.response,
                "metadata": metadata
            }
        return results, aggregated_results, rewritten_query

In [111]:
# Running the Extended Application
rag = RAGApplicationWithAgents()
print("Extended RAG Application Initialized")

query = "What are the key insights for Uber and Infosys in 2023?"
print("Running context-aware query...")



Initializing API keyss
API Keys verified
Embedding model initialized
Embedding model set
Initializing Pinecone index...
Pinecone manager initialized
Document manager initialized
Query engine manager initialized
Query rewriter initialized
Extended RAG Application Initialized
Running context-aware query...


In [116]:
query = "how uber devoped financially from 2020 to 2023"

original_results, results, rewritten_query = rag.run_context_aware_query(query)
print("Original Results:", original_results)
print("Rewritten Query:", rewritten_query)
print("Results:")
for index, result in results.items():
    print(f"Index: {index}")
    print(f"Response: {result['response']}")
    print(f"Metadata:{result["metadata"]}")
    print("Relevant Documents:")
    for doc in result["documents"]:
        print(doc)

Rewritten_query How did Uber's financial performance evolve from 2020 to 2023?
Querying index: uber-llama-parser
Original Results: {'uber-llama-parser': Response(response="Uber's financial performance improved significantly from 2020 to 2023. In 2020, Uber experienced losses, with a net loss attributable to Uber Technologies, Inc. of $6.92 billion. However, by 2023, Uber had turned around its financials, achieving a net income attributable to Uber Technologies, Inc. of $1.89 billion. This positive shift was driven by various factors such as revenue growth, increased Adjusted EBITDA, and favorable impacts from unrealized gains on debt and equity securities. Additionally, Uber's revenue increased from $11.14 billion in 2020 to $37.28 billion in 2023, showcasing a substantial growth in its financial performance over the years.", source_nodes=[NodeWithScore(node=TextNode(id_='12a36058-566e-42da-9faf-4746fd1953de', embedding=[-0.005377803, -0.02357429, 0.04254936, 0.010492793, 0.00067592115

In [120]:
results["uber-llama-parser"]["metadata"]

[{'name': 'uber-2023.pdf'},
 {'name': 'uber-2020.pdf'},
 {'name': 'uber-2020.pdf'},
 {'name': 'uber-2020.pdf'},
 {'name': 'uber-2023.pdf'},
 {'name': 'uber-2022.pdf'},
 {'name': 'uber-2023.pdf'}]

In [91]:
print(results["uber-llama-parser"]["documents"])
# print(results["infosys-llama-parser"]["documents"]) 
# print(results["alteryx-llama-parser"]["documents"]) 

['# Financial and Operational Highlights\n\n|Year Ended December 31,|2022|2023|% Change|(Constant Currency(1))|\n|---|---|---|---|---|\n|Monthly Active Platform Consumers (“MAPCs”)(2), (3)|131|150|15 %| |\n|Trips(2)|7,642|9,448|24 %| |\n|Gross Bookings(2)|$ 115,395|$ 137,865|19 %|20 %|\n|Revenue|$ 31,877|$ 37,281|17 %|18 %|\n|Income (loss) from operations|$ (1,832)|$ 1,110|**| |\n|Net income (loss) attributable to Uber Technologies, Inc.|$ (9,141)|$ 1,887|**| |\n|Adjusted EBITDA(1), (2)|$ 1,713|$ 4,052|137 %| |\n|Net cash provided by operating activities(4)|$ 642|$ 3,585|**| |\n|Free cash flow(1), (4)|$ 390|$ 3,362|**| |\n\n(1) See the section titled “Reconciliations of Non-GAAP Financial Measures” for more information and reconciliations to the most directly comparable GAAP financial measure.\n\n(2) See the section titled “Certain Key Metrics and Non-GAAP Financial Measures” below for more information.\n\n(3) MAPCs presented for annual periods are MAPCs for the fourth quarter of the y

In [79]:
len(results["uber-llama-parser"]["documents"])

7

In [107]:
original_results["uber-llama-parser"].source_nodes[6].metadata

{'name': 'uber-2023.pdf'}

In [106]:
print("Original Results:", original_results.keys())
original_results_uber = original_results["uber-llama-parser"]
# original_results_infosys = original_results["infosys-llama-parser"] 
# original_results_alteryx = original_results["alteryx-llama-parser"]
print("Original Results Uber:", original_results_uber)
# print("Original Results Infosys:", original_results_infosys)
# print("Original Results Alteryx:", original_results_alteryx)

Original Results: dict_keys(['uber-llama-parser'])
Original Results Uber: Uber's financial performance improved significantly from 2020 to 2023. In 2020, Uber experienced losses, with a net loss attributable to Uber Technologies, Inc. of $6.92 billion. However, by 2023, Uber had turned around its financials, achieving a net income attributable to Uber Technologies, Inc. of $1.89 billion. This positive shift was driven by various factors such as revenue growth, increased Adjusted EBITDA, and favorable impacts from unrealized gains on debt and equity securities. Additionally, Uber's revenue increased from $11.14 billion in 2020 to $37.28 billion in 2023, showcasing a substantial growth in its financial performance over the years.


In [23]:
query = "What companies you aware"
print("Running context-aware query...")

results, rewritten_query = rag.run_context_aware_query(query)
print("Rewritten Query:", rewritten_query)

Running context-aware query...


Querying index: uber-llama-parser
Querying index: infosys-llama-parser
Querying index: alteryx-llama-parser
Rewritten Query: Which companies are you familiar with?


In [25]:
print("Results:")
context = ""
for index, result in results.items():
    print(50*"*","Index: ",index,50*"*")
    print(f"Response: {result['response']}")
    print("Relevant Documents:")
    for doc in result["documents"]:
        context += "\n" + doc 
        print(doc)


Results:
************************************************** Index:  uber-llama-parser **************************************************
Response: I am familiar with companies such as Robinson, Total Quality Logistics, XPO Logistics, Convoy, Echo Global Logistics, Coyote, Transfix, D.H.L., and N.E.T. Trucking in the logistics industry.
Relevant Documents:
Robinson, Total Quality Logistics, XPO Logistics, Convoy, Echo Global Logistics, Coyote, Transfix, D.H.L., and N.E.T. Trucking.

In May 2020, we divested certain assets of our dockless e-bikes and scooters business to Lime and concurrently entered into a commercial partnership with Lime. In December 2020, we announced that we entered into a definitive agreement in connection with the ATG Transactions. In January 2021, we completed the sale of Apparate USA, Inc., our subsidiary focused on the development and commercialization of autonomous vehicles technologies, to Aurora Innovation, Inc. (“Aurora”) and made a $400 million cash investm

In [26]:
print(len(context))

74876


In [32]:
# Running the Application
rag = RAGApplication()
print("RAG Application Initialized")
input_files = ["./pdfs/alteryx-2023.pdf", "./pdfs/gitlab-2023.pdf", "./pdfs/infosys-2023.pdf", "./pdfs/uber-2023.pdf"
                "./pdfs/alteryx-2022.pdf", "./pdfs/gitlab-2022.pdf", "./pdfs/infosys-2022.pdf", "./pdfs/uber-2022.pdf",
                "./pdfs/alteryx-2021.pdf", "./pdfs/gitlab-2021.pdf", "./pdfs/infosys-2021.pdf", "./pdfs/uber-2021.pdf",
                "./pdfs/alteryx-2020.pdf", "./pdfs/infosys-2020.pdf", "./pdfs/uber-2020.pdf"]
uploaded_files = ["./pdfs/alteryx-2023.pdf", "./pdfs/gitlab-2023.pdf", "./pdfs/infosys-2023.pdf", "./pdfs/uber-2023.pdf"
                "./pdfs/alteryx-2022.pdf", "./pdfs/gitlab-2022.pdf", "./pdfs/infosys-2022.pdf", "./pdfs/uber-2022.pdf",
                "./pdfs/alteryx-2021.pdf", "./pdfs/gitlab-2021.pdf", "./pdfs/infosys-2021.pdf", "./pdfs/uber-2021.pdf",
                "./pdfs/alteryx-2020.pdf", "./pdfs/infosys-2020.pdf", "./pdfs/uber-2020.pdf"]

# rag.upload_documents(input_files)


Initializing API keyss
API Keys verified
Embedding model initialized
Embedding model set
Initializing Pinecone index...
Pinecone manager initialized
Document manager initialized
Query engine manager initialized
Query rewriter initialized
RAG Application Initialized


In [27]:
from dotenv import load_dotenv
from llama_index.llms.openai import OpenAI
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate

class OPENAI_LLM:
    def __init__(self):
        load_dotenv()
        OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
        if not OPENAI_API_KEY:
            raise ValueError("OpenAI API key is missing. Ensure it is set in the .env file.")

        self.llm = OpenAI(model="gpt-4o", temperature=0.1, api_key=OPENAI_API_KEY)

                

    def get_response(self, history, relevant_documents, query):
        context = ""
        for doc in relevant_documents:
            context += doc + "\n"

        qa_prompt = f"""
            You are an AI assisstent[you are aware of Infosys, Uber and Alteryx companies financial reports] that answers questions strictly based on the provided Relevants Documents and conversation history, if available.
            If no context is available or no context is used to answer, respond with 'No relevant information found.'

            ### Conversation History:
            {history}

            ### Relevant Documents:
            {context}

            ### User Query:
            {query}

            ### Guidelines for Answer:
            1. Provide the **Final Answer** concisely without displaying the step-by-step reasoning process or irrelevant details.
            2. If the user query does not specify a year, provide data for all available years mentioned in the context.
            3. If the user query does not specify a industry name, provide data for all available industries mentioned in the context 
            4. Include an **Explanation** of how the relevant documents helped in answering the query, focusing on why the context is relevant.
            5. If no relevant documents are found, explicitly state: 'No relevant information found.'
            6. If query context is out of the box, just say sorry and I am not aware about it.
        """
        print("*"*50,"QA Prompt: ", "*"*50,"\n", qa_prompt)

        chat_text_qa_msgs = [
            ChatMessage(
                role=MessageRole.ASSISTANT,
                content= "You are helpfull AI assisstent"
                ),
            ChatMessage(role=MessageRole.USER, content= qa_prompt),
        ]

         # Creating Chat Prompt Template
        text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)
        # print("Creating ChatPromptTemplate:", text_qa_template)

        #QA Response from formatted msgs
        response = text_qa_template.format_messages(chat_text_qa_msgs)
        # print("*"*50,"QA Response: ", "*"*50,"\n", qa_response)

        print(response)

        ai_response = self.llm.chat(response)

        return ai_response.message.blocks[0].text.strip()

In [34]:
history = ""
print("Running query...")
# relevant_documents, rewritten_query = rag.run_query(rewritten_query, history)
openai_response = OPENAI_LLM().get_response("", context, rewritten_query)

Running query...
************************************************** QA Prompt:  ************************************************** 
 
            You are an AI assisstent[you are aware of Infosys, Uber and Alteryx companies financial reports] that answers questions strictly based on the provided Relevants Documents and conversation history, if available.
            If no context is available or no context is used to answer, respond with 'No relevant information found.'

            ### Conversation History:
            

            ### Relevant Documents:
            

R
o
b
i
n
s
o
n
,
 
T
o
t
a
l
 
Q
u
a
l
i
t
y
 
L
o
g
i
s
t
i
c
s
,
 
X
P
O
 
L
o
g
i
s
t
i
c
s
,
 
C
o
n
v
o
y
,
 
E
c
h
o
 
G
l
o
b
a
l
 
L
o
g
i
s
t
i
c
s
,
 
C
o
y
o
t
e
,
 
T
r
a
n
s
f
i
x
,
 
D
.
H
.
L
.
,
 
a
n
d
 
N
.
E
.
T
.
 
T
r
u
c
k
i
n
g
.




I
n
 
M
a
y
 
2
0
2
0
,
 
w
e
 
d
i
v
e
s
t
e
d
 
c
e
r
t
a
i
n
 
a
s
s
e
t
s
 
o
f
 
o
u
r
 
d
o
c
k
l
e
s
s
 
e
-
b
i
k
e
s
 
a
n
d
 
s
c
o
o
t
e
r
s
 
b
u
s
i
n


In [35]:
openai_response

'### Final Answer:\nI am aware of the companies Infosys, Uber, and Alteryx.\n\n### Explanation:\nThe relevant documents provided information about the financial reports and business operations of Infosys, Uber, and Alteryx. These companies were mentioned in various contexts, including their subsidiaries, strategic partnerships, and market presence, which helped identify them as the companies I am aware of.'

In [31]:
from dotenv import load_dotenv
from llama_index.llms.openai import OpenAI
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate

class GEMINI_LLM:
    def __init__(self):
        load_dotenv()
        GEMINI_API_KEY = os.environ.get("GEMINI_API_KEY")
        if not GEMINI_API_KEY:
            raise ValueError("OpenAI API key is missing. Ensure it is set in the .env file.")

        genai.configure(api_key=GEMINI_API_KEY)
        self.llm = genai.GenerativeModel(model_name = "gemini-1.5-flash")
        print("Gemini is Created")   


    def get_response(self, history, relevant_documents, query):
        context = ""
        for doc in relevant_documents:
            context += doc + "\n"

        qa_prompt = f"""
            You are an AI assisstent[you are aware of Infosys, Uber and Alteryx companies financial reports] that answers questions strictly based on the provided Relevants Documents and conversation history, if available.
            If no context is available or no context is used to answer, respond with 'No relevant information found.'

            ### Conversation History:
            {history}

            ### Relevant Documents:
            {context}

            ### User Query:
            {query}

            ### Guidelines for Answer:
            1. Provide the **Final Answer** concisely without displaying the step-by-step reasoning process or irrelevant details.
            2. If the user query does not specify a year, provide data for all available years mentioned in the context.
            3. If the user query does not specify a industry name, provide data for all available industries mentioned in the context 
            4. Include an **Explanation** of how the relevant documents helped in answering the query, focusing on why the context is relevant.
            5. If no relevant documents are found, explicitly state: 'No relevant information found.'
            6. If query context is out of the box, just say sorry and I am not aware about it.
        """
        print("*"*50,"QA Prompt: ", "*"*50,"\n", qa_prompt)

        chat_text_qa_msgs = [
            ChatMessage(
                role=MessageRole.ASSISTANT,
                content= "You are helpyou AI assisstent"
                ),
            ChatMessage(role=MessageRole.USER, content= qa_prompt),
        ]

         # Creating Chat Prompt Template
        text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)
        # print("Creating ChatPromptTemplate:", text_qa_template)

        #QA Response from formatted msgs
        response = text_qa_template.format_messages(chat_text_qa_msgs)
        # print("*"*50,"QA Response: ", "*"*50,"\n", qa_response)

        print(response)

        ai_response = self.llm.generate_content(response[1].blocks[0].text)
        return ai_response.text.strip()

In [32]:
# query = "what is Infosys revenue in 2023?"
history = ""
print("Running query...")
relevant_documents, rewritten_query = rag.run_query(query, history)
gemini_response = GEMINI_LLM().get_response("", relevant_documents, rewritten_query)

Running query...
Running query on the query engine...
Gemini is Created
************************************************** QA Prompt:  ************************************************** 
 
            You are an AI assisstent[you are aware of Infosys, Uber and Alteryx companies financial reports] that answers questions strictly based on the provided Relevants Documents and conversation history, if available.
            If no context is available or no context is used to answer, respond with 'No relevant information found.'

            ### Conversation History:
            

            ### Relevant Documents:
            # Name of subsidiaries

|Country|Holdings as at March 31|2021|2020|
|---|---|---|---|
|Kallidus Inc, (Kallidus) (45)|US|–|100%|
|Infosys Chile SpA|Chile|100%|100%|
|Infosys Arabia Limited (2)|Saudi Arabia|70%|70%|
|Infosys Consulting Ltda. (1)|Brazil|100%|100%|
|Infosys CIS LLC (1) (19)|Russia|–|–|
|Infosys Luxembourg S.a.r.l|Luxembourg|100%|100%|
|Infosys Americas I

In [33]:
gemini_response

"**Final Answer:** Infosys, Uber, and Alteryx.\n\n**Explanation:** The provided text mentions Infosys in the context of its subsidiaries and financial information.  Uber is mentioned in a passage describing its competitive landscape and business activities. Alteryx is not explicitly named, but the question asks about companies the AI is *aware* of based on provided documents, and the prompt states the AI is aware of Alteryx's financial reports (although no such reports are included in the provided documents).  The answer is therefore based on the prompt's premise, not the provided documents themselves regarding Alteryx."